In [11]:
import sys
sys.path.insert(0, "/scratch/guanguowei/Code/MyWork/VIP5_Shadowcast_DPA")

In [12]:
import modeling_vip5
print(modeling_vip5.__file__)

import sys
sys.path.append('../')

import collections
import os
import re
import random
from pathlib import Path
import logging
import shutil
import time
from packaging import version
from collections import defaultdict

from tqdm import tqdm
import numpy as np
import gzip
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.backends.cudnn as cudnn

from src.param import parse_args
from src.utils import LossMeter
from src.dist_utils import reduce_dict
from transformers import T5Tokenizer
from src.tokenization import P5Tokenizer
from src.model import VIP5Tuning

_use_native_amp = False
_use_apex = False

# Check if Pytorch version >= 1.6 to switch between Native AMP and Apex
if version.parse(torch.__version__) < version.parse("1.6"):
    from transormers.file_utils import is_apex_available
    if is_apex_available():
        from apex import amp
    _use_apex = True
else:
    _use_native_amp = True
    from torch.cuda.amp import autocast

from src.trainer_base import TrainerBase

import pickle

def load_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)


def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
        
import json

def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)
    
def ReadLineFromFile(path):
    lines = []
    with open(path,'r') as fd:
        for line in fd:
            lines.append(line.rstrip('\n'))
    return lines

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

/scratch/guanguowei/Code/MyWork/VIP5_Shadowcast_DPA/notebooks/modeling_vip5.py


In [13]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self
        
args = DotDict()

args.distributed = False
args.multiGPU = True
args.fp16 = True
#####################
args.split = "toys"
#####################
args.train = args.split
args.valid = args.split
args.test = args.split
args.batch_size = 16
args.optim = 'adamw' 
args.warmup_ratio = 0.1
args.lr = 1e-3
args.num_workers = 4
args.clip_grad_norm = 5.0
args.losses = 'sequential,direct,explanation'
args.backbone = 't5-small'
#####################
args.image_feature_type = 'vitb32'
args.image_feature_size_ratio = 2
args.use_adapter = True
args.reduction_factor = 8
args.use_single_adapter = True
args.use_vis_layer_norm = True
args.add_adapter_cross_attn = True
args.use_lm_head_adapter = True
#####################
args.epoch = 20
args.local_rank = 0

args.comment = ''
args.train_topk = -1
args.valid_topk = -1
args.dropout = 0.1

args.tokenizer = 'p5'
args.max_text_length = 1024
args.gen_max_length = 64
args.do_lower_case = False

args.weight_decay = 0.01
args.adam_eps = 1e-6
args.gradient_accumulation_steps = 1

'''
Set seeds
'''
args.seed = 2022
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

'''
Whole word embedding & Category embedding
'''
args.whole_word_embed = True
args.category_embed = True

cudnn.benchmark = True
ngpus_per_node = torch.cuda.device_count()
args.world_size = ngpus_per_node

LOSSES_NAME = [f'{name}_loss' for name in args.losses.split(',')]
if args.local_rank in [0, -1]:
    print(LOSSES_NAME)
LOSSES_NAME.append('total_loss') # total loss

args.LOSSES_NAME = LOSSES_NAME

#####################
gpu = 0 # Change GPU ID
#####################
args.gpu = gpu
args.rank = gpu
print(f'Process Launching at GPU {gpu}')

torch.cuda.set_device('cuda:{}'.format(gpu))

comments = []
dsets = []
if 'toys' in args.train:
    dsets.append('toys')
if 'beauty' in args.train:
    dsets.append('beauty')
if 'sports' in args.train:
    dsets.append('sports')
if 'clothing' in args.train:
    dsets.append('clothing')
comments.append(''.join(dsets))
if args.backbone:
    comments.append(args.backbone)
comments.append(''.join(args.losses.split(',')))
if args.comment != '':
    comments.append(args.comment)
comment = '_'.join(comments)

from datetime import datetime
current_time = datetime.now().strftime('%b%d_%H-%M')

if args.local_rank in [0, -1]:
    run_name = f'{current_time}_GPU{args.world_size}'
    if len(comments) > 0:
        run_name += f'_{comment}'
    args.run_name = run_name
    print(args)

['sequential_loss', 'direct_loss', 'explanation_loss']
Process Launching at GPU 0
{'distributed': False, 'multiGPU': True, 'fp16': True, 'split': 'toys', 'train': 'toys', 'valid': 'toys', 'test': 'toys', 'batch_size': 16, 'optim': 'adamw', 'warmup_ratio': 0.1, 'lr': 0.001, 'num_workers': 4, 'clip_grad_norm': 5.0, 'losses': 'sequential,direct,explanation', 'backbone': 't5-small', 'image_feature_type': 'vitb32', 'image_feature_size_ratio': 2, 'use_adapter': True, 'reduction_factor': 8, 'use_single_adapter': True, 'use_vis_layer_norm': True, 'add_adapter_cross_attn': True, 'use_lm_head_adapter': True, 'epoch': 20, 'local_rank': 0, 'comment': '', 'train_topk': -1, 'valid_topk': -1, 'dropout': 0.1, 'tokenizer': 'p5', 'max_text_length': 1024, 'gen_max_length': 64, 'do_lower_case': False, 'weight_decay': 0.01, 'adam_eps': 1e-06, 'gradient_accumulation_steps': 1, 'seed': 2022, 'whole_word_embed': True, 'category_embed': True, 'world_size': 4, 'LOSSES_NAME': ['sequential_loss', 'direct_loss', '

In [14]:
image_feature_dim_dict = {
    'vitb32': 512,
    'vitb16': 512,
    'vitl14': 768,
    'rn50': 1024,
    'rn101': 512
}

def create_config(args):
    from transformers import T5Config
    from adapters import (
        AdapterController,
        OutputParallelAdapterLayer,
        AdapterConfig
    )

    if 't5' in args.backbone:
        config_class = T5Config
    else:
        return None

    config = config_class.from_pretrained(args.backbone)

    for k, v in vars(args).items():
        setattr(config, k, v)

    config.non_linearity = "relu"

    config.feat_dim = image_feature_dim_dict[args.image_feature_type]
    config.n_vis_tokens = args.image_feature_size_ratio
    config.use_vis_layer_norm = args.use_vis_layer_norm
    config.reduction_factor = args.reduction_factor

    config.use_adapter = args.use_adapter
    config.add_adapter_cross_attn = args.add_adapter_cross_attn
    config.use_lm_head_adapter = args.use_lm_head_adapter
    config.use_single_adapter = args.use_single_adapter

    config.dropout_rate = args.dropout
    config.dropout = args.dropout
    config.attention_dropout = args.dropout
    config.activation_dropout = args.dropout

    config.losses = args.losses

    tasks = re.split("[, ]+", args.losses) # tranform to list

    if args.use_adapter:
        CONFIG_CLASS = AdapterConfig

        config.adapter_config = CONFIG_CLASS()
        config.adapter_config.tasks = tasks
        config.adapter_config.d_model = config.d_model # for adapter
        config.adapter_config.use_single_adapter = args.use_single_adapter
        config.adapter_config.reduction_factor = args.reduction_factor
        config.adapter_config.track_z = False
    else:
        config.adapter_config = None

    return config


def create_tokenizer(args):
    from transformers import T5Tokenizer
    from src.tokenization import P5Tokenizer

    if 'p5' in args.tokenizer:
        tokenizer_class = P5Tokenizer

    tokenizer_name = args.backbone
    
    tokenizer = tokenizer_class.from_pretrained(
        tokenizer_name,
        max_length=args.max_text_length,
        do_lower_case=args.do_lower_case,
    )

    print(tokenizer_class, tokenizer_name)
    
    return tokenizer


def create_model(model_class, config=None):
    print(f'Building Model at GPU {args.gpu}')

    model_name = args.backbone

    model = model_class.from_pretrained(
        model_name,
        config=config
    )
    return model

In [15]:
config = create_config(args)

if args.tokenizer is None:
    args.tokenizer = args.backbone
    
tokenizer = create_tokenizer(args)

model_class = VIP5Tuning
model = create_model(model_class, config)

model = model.cuda()

if 'p5' in args.tokenizer:
    model.resize_token_embeddings(tokenizer.vocab_size)
    
model.tokenizer = tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


<class 'src.tokenization.P5Tokenizer'> t5-small
Building Model at GPU 0
JointEncoder initialized successfully.
T5Stack initialized successfully.


Some weights of VIP5Tuning were not initialized from the model checkpoint at t5-small and are newly initialized: ['decoder.block.1.layer.0.attn_adapter.adapters.direct.down_sampler.bias', 'encoder.block.5.layer.0.attn_adapter.adapters.direct.down_sampler.bias', 'decoder.block.4.layer.1.enc_attn_adapter.adapters.explanation.up_sampler.weight', 'encoder.visual_embedding.feat_embedding.0.model.0.weight', 'output_adapter.adapter.down_sampler.weight', 'decoder.block.3.layer.1.enc_attn_adapter.adapters.explanation.up_sampler.weight', 'encoder.block.1.layer.0.attn_adapter.adapters.direct.up_sampler.weight', 'encoder.block.1.layer.1.ff_adapter.adapters.direct.down_sampler.bias', 'encoder.block.0.layer.1.ff_adapter.adapters.explanation.down_sampler.weight', 'decoder.block.2.layer.1.enc_attn_adapter.adapters.sequential.up_sampler.bias', 'decoder.block.2.layer.2.ff_adapter.adapters.explanation.up_sampler.weight', 'decoder.block.3.layer.1.enc_attn_adapter.adapters.sequential.up_sampler.weight', 'd

lm_head initialized successfully.
OutputParallelAdapterLayer initialized successfully.
AdapterConfig: AdapterConfig(add_layer_norm_before_adapter=False, add_layer_norm_after_adapter=False, non_linearity='gelu_new', reduction_factor=8)


#### Load Model

In [17]:
args.load = "/scratch/guanguowei/Code/MyWork/VIP5_Shadowcast_DPA/snap/toys/0304/toys-vitb32-2-8-20/BEST_EVAL_LOSS.pth"  

# Load Checkpoint
from src.utils import load_state_dict, LossMeter, set_global_logging_level
from pprint import pprint

def load_checkpoint(ckpt_path):
    state_dict = load_state_dict(ckpt_path, 'cpu')
    results = model.load_state_dict(state_dict, strict=False)
    pprint(results)

ckpt_path = args.load
load_checkpoint(ckpt_path)

from src.all_templates import all_tasks as task_templates

_IncompatibleKeys(missing_keys=['output_adapter.adapter.down_sampler.weight', 'output_adapter.adapter.down_sampler.bias', 'output_adapter.adapter.up_sampler.weight', 'output_adapter.adapter.up_sampler.bias'], unexpected_keys=[])


#### Check Test Split

In [18]:
data_splits = load_pickle('../data/{}/rating_splits_augmented.pkl'.format(args.split))
test_review_data = data_splits['test']

In [19]:
len(test_review_data)

16759

In [20]:
test_review_data[0]

{'reviewerID': 'A5K3CK2PWYQ7O',
 'asin': 'B00F4CFEYG',
 'reviewerName': 'Ellie "mittbooks"',
 'helpful': [0, 0],
 'reviewText': "I've found the Melissa & Doug brand to be overall good, although there are occasional negatives.  This is definitely one of the toys we'll mark a &#34;winner.&#34;  The vacuum comes in two pieces that require minimal assembly (the long handle and the base need to be put together - no tools required).  The height is perfect for our two year old who is 3 feet tall.  The top part moves at about a 45 degree angle to facilitate little people pushing the vacuum.  I'm not sure how long the six wooden pieces of &#34;trash&#34; will last.  Although not tiny, they would be easy to lose.  The vacuum does a good job of picking them up easily and there is a small area in the back of the base to take them out again.  There is also a rotating knob on the front of the handle that makes a good clicking noise when it moves.  Our son is truly enjoying this this toy and the over

In [21]:
data_maps = load_json(os.path.join('../data', args.split, 'datamaps.json'))
print(len(data_maps['user2id'])) # number of users
print(len(data_maps['item2id'])) # number of items

19412
11924


### Test P5

In [22]:
from torch.utils.data import DataLoader, Dataset, Sampler
from src.data import get_loader
from evaluate.utils import rouge_score, bleu_score, unique_sentence_percent, root_mean_square_error, mean_absolute_error, feature_detect, feature_matching_ratio, feature_coverage_ratio, feature_diversity
from evaluate.metrics4rec import evaluate_all

#### Evaluation - Explanation

In [23]:
import os
print(os.path.abspath("data/toys/exp_splits.pkl"))

/scratch/guanguowei/Code/MyWork/VIP5_Shadowcast_DPA/notebooks/data/toys/exp_splits.pkl


使用 importlib.reload 强制刷新 确保 Python 加载到你新修改过的 get_loader（带 data_root 参数）。

In [24]:
import importlib
import src.data
importlib.reload(src.data)
from src.data import get_loader

In [25]:
test_task_list = {'explanation': ['C-12']
}
test_sample_numbers = {'sequential': (1, 1), 'direct': (1, 1), 'explanation': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed,
        data_root="../data",        # 显式指明数据在上级目录
        feature_root="../features"  # 依此类推
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
BLEU1 = bleu_score(tokens_test, tokens_predict, n_gram=1, smooth=False)
print('BLEU-1 {:7.4f}'.format(BLEU1))
BLEU4 = bleu_score(tokens_test, tokens_predict, n_gram=4, smooth=False)
print('BLEU-4 {:7.4f}'.format(BLEU4))

ROUGE = rouge_score(tokens_test, tokens_predict)  # a dictionary
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


Data sources:  ['toys']
compute_datum_info
646


0it [00:00, ?it/s]/scratch/guanguowei/miniconda3/envs/vip5_env/lib/python3.9/site-packages/transformers/modeling_utils.py:810: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/scratch/guanguowei/miniconda3/envs/vip5_env/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/scratch/guanguowei/miniconda3/envs/vip5_env/lib/python3.9/site-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
646it [01:36,  6.68it/s]


BLEU-1 12.4267
BLEU-4  7.8793
rouge_1/f_score 27.2064
rouge_1/r_score 22.2341
rouge_1/p_score 44.9808
rouge_2/f_score  8.7732
rouge_2/r_score  7.3634
rouge_2/p_score 15.3068
rouge_l/f_score 20.7347
rouge_l/r_score 20.4198
rouge_l/p_score 41.7443


In [26]:
test_task_list = {'explanation': ['C-3']
}
test_sample_numbers = {'sequential': (1, 1), 'direct': (1, 1), 'explanation': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed,
        data_root="../data",        # 显式指明数据在上级目录
        feature_root="../features"  # 依此类推
)
print(len(zeroshot_test_loader))

tokens_predict = []
tokens_test = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        tokens_predict.extend(results) 
        tokens_test.extend(batch['target_text'])
        
BLEU1 = bleu_score(tokens_test, tokens_predict, n_gram=1, smooth=False)
print('BLEU-1 {:7.4f}'.format(BLEU1))
BLEU4 = bleu_score(tokens_test, tokens_predict, n_gram=4, smooth=False)
print('BLEU-4 {:7.4f}'.format(BLEU4))

ROUGE = rouge_score(tokens_test, tokens_predict)  # a dictionary
for (k, v) in ROUGE.items():
    print('{} {:7.4f}'.format(k, v))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


Data sources:  ['toys']
compute_datum_info
646


646it [01:27,  7.36it/s]


BLEU-1  2.9930
BLEU-4  1.0293
rouge_1/f_score  6.7310
rouge_1/r_score  5.1061
rouge_1/p_score 13.6365
rouge_2/f_score  1.2484
rouge_2/r_score  1.0052
rouge_2/p_score  2.3944
rouge_l/f_score  5.0368
rouge_l/r_score  4.8688
rouge_l/p_score 13.2084


#### Evaluation - Sequential

In [27]:
test_task_list = {'sequential': ['A-9']
}
test_sample_numbers = {'sequential': (1, 1), 'direct': (1, 1), 'explanation': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed,
        data_root="../data",        # 显式指明数据在上级目录
        feature_root="../features"  # 依此类推
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                input_ids=batch['input_ids'].to('cuda'), 
                whole_word_ids=batch['whole_word_ids'].to('cuda'), 
                category_ids=batch['category_ids'].to('cuda'), 
                vis_feats=batch['vis_feats'].to('cuda'), 
                task=batch["task"][0],
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


Data sources:  ['toys']
compute_datum_info
1214


1214it [20:38,  1.02s/it]



NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0609	0.0705	0.0705	0.0141	0.0577	0.0577

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.0637	0.0792	0.0792	0.0079	0.0588	0.0588


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.0637\t0.0792\t0.0792\t0.0079\t0.0588\t0.0588',
 {'ndcg': 0.06367775589005802,
  'map': 0.05883500714661851,
  'recall': 0.0791778281475376,
  'precision': 0.007917782814753532,
  'mrr': 0.05883500714661851,
  'hit': 0.0791778281475376})

In [28]:
test_task_list = {'sequential': ['A-3']
}
test_sample_numbers = {'sequential': (1, 1), 'direct': (1, 1), 'explanation': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed,
        data_root="../data",        # 显式指明数据在上级目录
        feature_root="../features"  # 依此类推
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                input_ids=batch['input_ids'].to('cuda'), 
                whole_word_ids=batch['whole_word_ids'].to('cuda'), 
                category_ids=batch['category_ids'].to('cuda'), 
                vis_feats=batch['vis_feats'].to('cuda'), 
                task=batch["task"][0],
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


Data sources:  ['toys']
compute_datum_info
1214


1214it [20:27,  1.01s/it]



NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0623	0.0724	0.0724	0.0145	0.0589	0.0589

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.0650	0.0806	0.0806	0.0081	0.0600	0.0600


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.0650\t0.0806\t0.0806\t0.0081\t0.0600\t0.0600',
 {'ndcg': 0.06495384704007565,
  'map': 0.060037564720235234,
  'recall': 0.08062023490624357,
  'precision': 0.00806202349062412,
  'mrr': 0.060037564720235234,
  'hit': 0.08062023490624357})

#### Evaluation - Direct

In [29]:
test_task_list = {'direct': ['B-8']
}
test_sample_numbers = {'sequential': (1, 1), 'direct': (1, 1), 'explanation': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed,
        data_root="../data",        # 显式指明数据在上级目录
        feature_root="../features"  # 依此类推
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                input_ids=batch['input_ids'].to('cuda'), 
                whole_word_ids=batch['whole_word_ids'].to('cuda'), 
                category_ids=batch['category_ids'].to('cuda'), 
                vis_feats=batch['vis_feats'].to('cuda'), 
                task=batch["task"][0],
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 1)
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


Data sources:  ['toys']
compute_datum_info
1214


1214it [30:36,  1.51s/it]



NDCG@1	Rec@1	Hits@1	Prec@1	MAP@1	MRR@1
0.0416	0.0416	0.0416	0.0416	0.0416	0.0416

NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0828	0.1231	0.1231	0.0246	0.0696	0.0696

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.1034	0.1874	0.1874	0.0187	0.0779	0.0779


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.1034\t0.1874\t0.1874\t0.0187\t0.0779\t0.0779',
 {'ndcg': 0.10335083045647782,
  'map': 0.07793226003224972,
  'recall': 0.1874098495775809,
  'precision': 0.01874098495775882,
  'mrr': 0.07793226003224972,
  'hit': 0.1874098495775809})

In [ ]:
test_task_list = {'direct': ['B-5']
}
test_sample_numbers = {'sequential': (1, 1), 'direct': (1, 1), 'explanation': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='test', 
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed,
        data_root="../data",        # 显式指明数据在上级目录
        feature_root="../features"  # 依此类推
)
print(len(zeroshot_test_loader))

all_info = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        beam_outputs = model.generate(
                input_ids=batch['input_ids'].to('cuda'), 
                whole_word_ids=batch['whole_word_ids'].to('cuda'), 
                category_ids=batch['category_ids'].to('cuda'), 
                vis_feats=batch['vis_feats'].to('cuda'), 
                task=batch["task"][0],
                max_length=50, 
                num_beams=20,
                no_repeat_ngram_size=0, 
                num_return_sequences=20,
                early_stopping=True
        )
        generated_sents = model.tokenizer.batch_decode(beam_outputs, skip_special_tokens=True)
        for j, item in enumerate(zip(results, batch['target_text'], batch['source_text'])):
            new_info = {}
            new_info['target_item'] = item[1]
            new_info['gen_item_list'] = generated_sents[j*20: (j+1)*20]
            all_info.append(new_info)
            
gt = {}
ui_scores = {}
for i, info in enumerate(all_info):
    gt[i] = [int(info['target_item'])]
    pred_dict = {}
    for j in range(len(info['gen_item_list'])):
        try:
            pred_dict[int(info['gen_item_list'][j])] = -(j+1)
        except:
            pass
    ui_scores[i] = pred_dict
    
evaluate_all(ui_scores, gt, 1)
evaluate_all(ui_scores, gt, 5)
evaluate_all(ui_scores, gt, 10)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'P5Tokenizer'.


Data sources:  ['toys']
compute_datum_info
1214


1214it [30:36,  1.51s/it]



NDCG@1	Rec@1	Hits@1	Prec@1	MAP@1	MRR@1
0.0409	0.0409	0.0409	0.0409	0.0409	0.0409

NDCG@5	Rec@5	Hits@5	Prec@5	MAP@5	MRR@5
0.0834	0.1249	0.1249	0.0250	0.0698	0.0698

NDCG@10	Rec@10	Hits@10	Prec@10	MAP@10	MRR@10
0.1057	0.1947	0.1947	0.0195	0.0789	0.0789


('\nNDCG@10\tRec@10\tHits@10\tPrec@10\tMAP@10\tMRR@10\n0.1057\t0.1947\t0.1947\t0.0195\t0.0789\t0.0789',
 {'ndcg': 0.10571562563801302,
  'map': 0.07886802550541473,
  'recall': 0.19467339789820728,
  'precision': 0.01946733978982163,
  'mrr': 0.07886802550541473,
  'hit': 0.19467339789820728})

: 